In [78]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
nltk.download('stopwords')
stopwords = stopwords.words('english')
remove_terms = ['the', '``', "''", "'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would']
stopwords = stopwords + remove_terms + list(string.punctuation)
print(stopwords)
lemmatizer = nltk.WordNetLemmatizer()

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/carlostezna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [115]:
wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', 
                                                                binary = True)

In [11]:
def load_document(file):
    f = open(file)
    try:
        raw = f.read()
        return file.split('/')[-1], raw
    except:
        print(file)
        pass

def load_collection(files):
    texts = []
    for file in files:
        doc_id, text = load_document(file)
        texts.append(text)
    return texts

In [118]:
import os
COLLECTION_DIR = './dataset/newDataset/'
files = [COLLECTION_DIR + file for file in os.listdir(COLLECTION_DIR)]

corpus = load_collection(files)

In [119]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# using default tokenizer in TfidfVectorizer
tfidf = TfidfVectorizer(token_pattern='(\S+)', min_df=3, max_df=0.8, 
                        ngram_range=(1, 2), stop_words=stopwords, tokenizer=word_tokenize)
features = tfidf.fit_transform(corpus)
df = pd.DataFrame(
        features.todense(),
        columns=tfidf.get_feature_names()
    )

In [120]:
len(corpus)

2135

In [121]:
df.head()

,'best,'bot,'bot nets,'cold,'cold calls,'could,'do,'eu,'goodbye,'goodbye said,...,£8.5bn,£800m,£800m 1.5bn,£80m,£857m,£8bn,£8m,£9.4m,£99,£9m
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
len(tfidf.vocabulary_)

31809

In [123]:
tfidf.vocabulary_

{'custody': 7252,
 'death': 7528,
 'rate': 22504,
 'mps': 18232,
 'deaths': 7531,
 'reached': 22575,
 'shocking': 25604,
 'levels': 15940,
 'committee': 6114,
 'peers': 20445,
 'warned': 30431,
 'joint': 14870,
 'human': 13633,
 'rights': 23542,
 'found': 11228,
 'committing': 6129,
 'suicide': 27282,
 'mainly': 16765,
 'vulnerable': 30268,
 'mental': 17554,
 'health': 12995,
 'drugs': 8650,
 'alcohol': 1825,
 'problems': 21818,
 'members': 17509,
 'urged': 29637,
 'government': 12361,
 'set': 25383,
 'task': 27728,
 'force': 11088,
 'tackle': 27581,
 'prisons': 21761,
 'police': 21255,
 'cells': 5179,
 'detention': 7964,
 'centres': 5208,
 'special': 26406,
 'hospitals': 13491,
 'one': 19523,
 'prison': 21755,
 'every': 9717,
 'four': 11250,
 'days': 7470,
 '1999': 452,
 '2003': 540,
 'report': 23170,
 'followed': 11035,
 'year-long': 31523,
 'inquiry': 14207,
 'high': 13179,
 'amounts': 2272,
 'serious': 25298,
 'failure': 10147,
 'protect': 22016,
 'right': 23511,
 'life': 15998,
 '

In [124]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(corpus)
print(finder.nbest(bigram_measures.pmi, 10))
finder.apply_freq_filter(10)

[('&', '#'), ('!', '!'), ('=', '5'), ('2', '='), ('(', '£'), ('#', '2'), ('U', 'K'), ('9', '9'), ('1', '9'), ('?', '?')]


In [125]:
def preprocess(document):
        """
        Tokenizes documents then normalizes and lemmatizes tokens
        """
        from nltk.tokenize import word_tokenize
        words = word_tokenize(document)
        words_clean = []
        for word in words: # Go through every word in your tokens list
            w = word.lower()
            if (w not in stopwords):  # remove stopwords and punctuation
                words_clean.append(lemmatizer.lemmatize(w))
        return words_clean

In [126]:
import gensim
from gensim import corpora, models
dictionary = corpora.Dictionary(preprocess(doc) for doc in corpus)
bow = [dictionary.doc2bow(preprocess(doc)) for doc in corpus]

In [130]:
ldamodel = models.ldamodel.LdaModel(bow, num_topics=20, id2word=dictionary, passes=50, minimum_probability=0.1)

[(0, '0.025*"wale" + 0.018*"england" + 0.018*"zealand" + 0.017*"new" + 0.014*"game"'), (1, '0.023*"band" + 0.019*"music" + 0.017*"best" + 0.016*"award" + 0.013*"album"'), (2, '0.028*"game" + 0.020*"said" + 0.015*"china" + 0.013*"lending" + 0.012*"sony"'), (3, '0.017*"ireland" + 0.012*"j" + 0.011*"minute" + 0.009*"g" + 0.009*"o\'gara"'), (4, '0.015*"said" + 0.014*"show" + 0.011*"people" + 0.009*"bbc" + 0.009*"u"'), (5, '0.015*"said" + 0.010*"system" + 0.010*"mobile" + 0.010*"technology" + 0.009*"people"'), (6, '0.021*"drug" + 0.016*"test" + 0.014*"sport" + 0.014*"also" + 0.014*"greek"'), (7, '0.021*"said" + 0.018*"michael" + 0.018*"film" + 0.011*"different" + 0.010*"life"'), (8, '0.022*"film" + 0.013*"number" + 0.012*"one" + 0.009*"year" + 0.007*"award"'), (9, '0.010*"said" + 0.010*"game" + 0.009*"player" + 0.008*"time" + 0.006*"world"'), (10, '0.017*"said" + 0.011*"program" + 0.009*"microsoft" + 0.008*"virus" + 0.007*"software"'), (11, '0.015*"said" + 0.009*"year" + 0.007*"carry" + 0.0

In [132]:
print(ldamodel.print_topics(num_topics=1, num_words=15))

[(18, '0.033*"said" + 0.012*"police" + 0.011*"new" + 0.011*"law" + 0.010*"mr" + 0.009*"lord" + 0.009*"power" + 0.009*"right" + 0.008*"court" + 0.007*"government" + 0.007*"case" + 0.007*"plan" + 0.007*"home" + 0.007*"human" + 0.006*"people"')]


In [141]:
analogy = wv_embeddings.most_similar(positive=['man', 'queen'], negative=['woman'])
print(analogy)

[('king', 0.6958590149879456), ('kings', 0.5950952768325806), ('queens', 0.5838501453399658), ('monarch', 0.5398427248001099), ('prince', 0.5223615169525146), ('princess', 0.5175285339355469), ('princes', 0.49844634532928467), ('royal', 0.4924592971801758), ('NYC_anglophiles_aflutter', 0.4859851002693176), ('Eugene_Ionesco_absurdist_comedy', 0.4784241318702698)]


In [157]:
similar_word = wv_embeddings.most_similar('shit')
print(similar_word)

[('sh_*_t', 0.8806686401367188), ('crap', 0.8299816250801086), ('fucking', 0.8001799583435059), ('sh_t', 0.7744057178497314), ('fuckin', 0.7668500542640686), ('sh_**', 0.7638066411018372), ('fuck', 0.7604621648788452), ('f_*_cking', 0.7250384092330933), ("y'know", 0.7228368520736694), ('shyt', 0.7212573885917664)]


In [209]:
import re
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases

In [175]:
# Cleaning data - remove punctuation from every text
texts = corpus.copy()
sentences = []
# Go through each text in turn
for ii in range(len(texts)):
    sentences = [re.sub(pattern=r'[\!"#$%&\*+,-./:;<=>?@^_`()|~=]', 
                        repl='', 
                        string=x
                       ).strip().split(' ') for x in texts[ii].split('\n') 
                      if not x.endswith('writes:')]
    sentences = [x for x in sentences if x != ['']]
    texts[ii] = sentences

In [178]:
# concatenate all sentences from all texts into a single list of sentences
all_sentences = []
for text in texts:
    all_sentences += text

In [179]:
# Phrase Detection
# Give some common terms that can be ignored in phrase detection
# For example, 'state_of_affairs' will be detected because 'of' is provided here: 
common_terms = ["of", "with", "without", "and", "or", "the", "a"]
# Create the relevant phrases from the list of sentences:
phrases = Phrases(all_sentences, common_terms=common_terms)
# The Phraser object is used from now on to transform sentences
bigram = Phraser(phrases)
# Applying the Phraser to transform our sentences is simply
all_sentences = list(bigram[all_sentences])

In [234]:
import time
start = time.time()
model = Word2Vec(all_sentences, 
                 min_count=3,   # Ignore words that appear less than this
                 size=200,      # Dimensionality of word embeddings
                 workers=8,     # Number of processors (parallelisation)
                 window=5,      # Context window for words during training
                 iter=100)       # Number of epochs training over corpus
end = time.time()
print('Train time: ', end - start)

Train time:  48.43529510498047


In [248]:
model.wv.most_similar('Google')

[('Yahoo', 0.49438562989234924),
 ('search', 0.4441230893135071),
 ('web', 0.441272497177124),
 ('search_engine', 0.4357411861419678),
 ('MSN', 0.40594184398651123),
 ('Microsoft', 0.4045748710632324),
 ('AOL', 0.3857986330986023),
 ("Microsoft's", 0.3629482686519623),
 ('Ask_Jeeves', 0.36271706223487854),
 ('site', 0.35542720556259155)]